# Clause Parser Algorithm

In [1]:
!python --version
!python -m spacy download en_core_web_lg

print("Downloaded")

Python 3.6.3
    100% |████████████████████████████████| 852.3MB 27.6MB/s ta 0:00:01 0% |                                | 1.3MB 3.8MB/s eta 0:03:47    3% |█                               | 27.2MB 4.0MB/s eta 0:03:29    13% |████▏                           | 112.3MB 1.4MB/s eta 0:09:02    18% |█████▉                          | 155.9MB 1.2MB/s eta 0:09:33    21% |██████▉                         | 181.1MB 3.7MB/s eta 0:03:00    30% |█████████▊                      | 257.8MB 6.7MB/s eta 0:01:29    30% |█████████▉                      | 261.1MB 5.8MB/s eta 0:01:42    32% |██████████▍                     | 277.6MB 4.3MB/s eta 0:02:15    42% |█████████████▊                  | 364.6MB 15.4MB/s eta 0:00:32��██▍               | 436.5MB 13.8MB/s eta 0:00:31    52% |████████████████▊               | 446.1MB 9.8MB/s eta 0:00:42    59% |███████████████████             | 508.2MB 4.3MB/s eta 0:01:20    60% |███████████████████▎            | 512.3MB 27.9MB/s eta 0:00:13    60% |███████████████████▎   

In [2]:
import spacy
import html
from spacy import displacy

nlp = spacy.load('en_core_web_lg')
print("Loaded models")

Loaded models


In [3]:
from random import shuffle
def print_sample_list(arr):
    indices = [i for i in range(len(arr))]
    shuffle(indices)
    for x in indices[:min(5, len(arr))]:
        print(x, arr[x])

In [4]:
from io import StringIO
import pandas as pd, numpy as np

tsv = StringIO("""tempID	sentence
1	I am loving.
2	I am a loving person.
3	I am loving this sandwich.
4	In the beginning it was easy, but we aren't beginning anymore.
5	I like surfing.
6	You have a unique way of singing.
7	Meeting others can be an isolating or an enriching experience.
8	I have a suggestion to offer.
9	I wouldn't want you to think I can't diagram sentences.
10	I act in service to growing and helping others.
11	The car was wrecked in the crash.
12	The wrecked car was given to a junkyard.
13	Painting can be fun.
14	I like painting.
15	I have cleaned the room.
16	I have a clean room.
17	The room had been cleaned before we arrived.
18	I have a vacuum cleaner.
19	Do you have the time?
20	What a great time we had.
21	We would have had better luck if we had gone yesterday.
22	I have great luck at gambling.
23	He had trouble holding on to his money.
24	I have trouble understanding why he gambled so much.
25	I've never been here before.
26	I've a feeling we've been here before.
27	I've a good map in my pocket.
28	Sometimes I wish I could be younger and have done things differently.
29	That building is a mine.
30	That building is mine.
31	Mine's the one with the stripes.
32	My oh my, what a wonderful day!
33	That's my umbrella.
34	In my case, the officer waived the fine.
35	In that case, you were lucky.
36	The car's tires were stolen.
37	The car's not drivable without tires.
38	That is her doll.
39	That is my doll.
40	That car is theirs.
41	That car is mine.
42	Her way of doing things is wrong.
43	That is my way of doing it.
44	Can I go to the party?
45	I can't say why that is the case.
46	You can have dessert if you finish eating your supper.
47	That can be a reason to finish eating supper.
48	If you can, then say so.
49	I know I can.
50	I will not be ready in time.
51	His will must be obeyed.
52	I will answer for myself.
53	You might say it is a matter of might, or you might not.
54	You must eat your vegetables.
55	What shall we say to him about his behavior at mealtime?
56	We ought to let him know he needs to eat properly.
57	I get angry when you call me that.
58	I sometimes get confused about parts of speech.
59	I am often confused about parts of speech.
60	Can you please get me the manual on grammar?
61	I don't get why he gets so confused.
62	Listening to the radio gets me through the day.
63	I worried about not being able to get back home.
64	When the going gets tough, the tough get going.
65	It seems like a mistake to call that a verb.
66	She seems distracted.
67	I feel sick today.
68	I felt bad for her.
69	I feel nervous talking to strangers.
70	I felt safe, knowing that I could receive a warm welcome from my family and friends.
71	I let go of the rope.
72	I let the matter go.
73	Don't be sad.
74	Are you being serious?
75	I am a human being.
76	I may not be able to finish my lunch.
77	He was being foolish and paid the price.
78	My main problem is being too accommodating.
79	The act of being kind is valued.
80	Does this dress make me look fat?
81	Do you think we should tell him not to answer that?
82	How do you answer that?
83	How do you do?
84	The doing gets easier as your skill improves.
85	What does that machine do?
86	That's part of what it takes to succeed.
87	What do you make of this answer?
88	The part of the house where you enter is the doorway.
1001	Sometimes I wish that my mom would come home more often.
1002	My family has a cat...I don't have a cat though I have 2 dogs
1003	When I am nervous my hands get shaky
1004	At times I worry about my own dreams, nightmares I have thousands of them
1005	Technology its crazy and i wish i could keep up
1006	When I get mad I sometimes use big voice then I go somewhere quiet so I can cry by myself.
1007	My main problem is a feeling that I am not entitled
1008	If my mother was a different lady, I would not be who I am.
1009	When I get mad it takes a long time to build up, but I am slow to forgive.
1010	When they avoided me I felt safe, knowing that I could receive a warm welcome from my family and friends.
1011	Sometimes I wish that I could be younger and have done things differently.
1012	When they avoided me I sought to find out 'why?' . Knowing the 'why' can begin Awareness and Healing
1013	Being with other people is both a learning experience and an opportunity to exchange views.
1014	My main problem is only a main problem if I think it is.
1015	When people are helpless At times I try to find other ways of doing thing. But sometime I might be disappointed and stop or quit.
1016	My father taught me how to think critically and to love deeply.
1017	If I can\'t get what I want I try to work out another way of getting what I want
1018	When they avoided me hmmm... I\'m stumped. I have no idea how to respond to this... Although there is a sense of relief...
1019	Loving other people can deepen with familiarity. Even with those who aren't a natural fit the differences between us can add color to the world as well as an opportunity to turn inward to see what is being triggered. Having said that, it's difficult for me to open my heart and trust.
1020	When people are helpless I am inspired to reach out and be of service.
1021	Rules are guard rails. It is good and helpful to have them. When you head in the right direction you never get in touch with them, no matter how fast or slow you travel. Good rules are not in your way.
1022	Technology is a great tool, nice to have and can bring global humanity closer together in ways never before imagined. But if it were to disappear tomorrow, I'm of the generation that would be able to survive.
1023	A good boss A good boss is critical toward making a workplace environment healthy and successful!
1024	My main problem is getting over decades spent proving myself.
1025	I feel sorry for the pain that all humans and all beings on this earth must endure as part of the experience of being here.
1026	If I were in charge i would change that by facilitating change that made everyone in charge ... to the degree of their capabilities in wordly matters, but always in charge of their interior and their right to decide for themselves what is real and true for them.
1027	At times I worry about big, big existential questions, and little microscopic ones... sometimes within milliseconds of one another.
1028	Raising a family is a profound gift. I am tending the future. I am tending the life processes. It is a place for love and life and evolution to soar.
1029	At times I worry about how differently I see and process things from many of the people I encounter, and whether I am wisely acting and interacting.
1030	What I like to do best is be a conduit. There is no I.
1031	Being with other people can be an isolating or a hugely enriching experience, the latter when each others presence liberates emergent thoughts or self expressions bringing forth a collective that is more than the sum of its parts
1032	A teacher has the right to I don't mean to be oppositional. . but this sentence stem. . .like so many others. . isn't one I can really answer outside of context. Rights. This is a wobbly construct. I'm sorry. I'm out of time otherwise I would write a bit about the possibility of rights . .the idea of having a right. . the usage of the language of rights.
1033	Being with other people is most often a pleasure and fascinating experience of seeing a completely different version of Self.
1034	The past is a construct to give form and structure to the present moment.
1035	We could make the world a better place if the mystery was okay to be as a question, that is to say if the question was okay to be a question and the answer could be a question and the answers and questions could be held together because they need each other and we realize a little faster that they won't end and they are growing together and rest in that.
1036	A good boss is a solar being ... one that brings light and eternal love without demanding anything of the others ... an energetic and gravitational field of such calm, peaceful magnitude that all are warmed and captivated by their glow ... brings a new burst life to all those who are in their presence
1037	My conscience bothers me if I'm not placing my feet on the path with an open heart, mind and spirit, tuning in to what's moving and showing up fully at the dance.
1038	When I am criticized I can embody my personality and feel a pang contrast to the story I tell myself inside the story we tell ourselves of what we think in the moment to be an absolute before it falls away into a deeper, clearer calm of letting go.
1039	The thing I like about myself is that here I sit in the drop of this essential moment, chuckling at this question! :-D
1040	My main problem is integrating emptiness with everyday life, finding meaning in meaningless given the meaningless in meaning making.
1041	My main problem is how my narccissim gets in the way of seeing the truth of the illusion of self.
1042	These days, work Is a rainbow of consciousness manifesting through each sparkling emanation called a 'me'.
1043	Being with other people Is a mutually mutating meeting of universes
1044	What I like to do best is lay back and live the window that I have become as if a home with no walls, the infinite moves through as if there is no solid boundary.the veil is no more and the whole field that is everywhere exists where the inside and outside have no difference.
1045	Change is the heartbeat of the universe... aliveness calling us forth!
1046	When I get mad i feel adrenaline through my human body and it is fierce and i love it because it moves me and it is no different than the Spirit which moves all.
1047	Raising a family Captivates my whole being, just as every window on reality sees it is held in Love beyond knowing.
1048	If I were in charge Is an expression of the love animating all creation, the simple joy that moves all from rock to life to mind to All. It comes through this glorious One in all its parts and territories, from an inaccessible Heart, within, without.
1049	Privacy the human formconsicousness separates from the One even for a moment.
2001	Overlay must be done thoughtfully!    :)
2002	Less red tape should be the primary focus!
2003	More design flexibility. Less consultation.
2004	Totally against densification because it keeps the fire department too busy.
2005	Allow garage suite and garden suite size to increase in proportion to lot size
2006	The builders should be responsible for any damage to the neighboring properties.
2007	Siding should not be vinyl but instead a fire-retarding siding should be in place.
2008	sizes should be similar to existing homes- in square footage- footprint and height
2009	The consultation period should be at least 60 days from the time notice of the application is mailed to 60m surrounding neighbours or signage posted on the site.
2010	I worry about how infill will make our mature neighbourhoods ugly and take away from the spaciousness and beauty. Infill is ugly, no matter how pretty the building.
2011	Boulevard trees should not be removed.  This should be kept in mind when a home is being designed/approved. An Arborist needs to be consulted before a tree is removed.
2012	Once regulations are in place, they should be enforced!  It seems variances and exceptions are the rule to planning regulations.  IF there is an accepted plan, stick to it.
2013	mature neighbourhoods should be be designed to be more attractive than far-away suburbs with quality of life, walkable/cycling infrastructure and a feel similar to some european cities. Perhaps encourage coffee shops or other facilities to encourage the people friendly aspects.
2014	mature neighbourhoods should be be designed to be more attractive than far-away suburbs with quality of life, walkable/cycling infrastructure and a feel similar to some european cities. Perhaps encourage coffee shops or other facilities to encourage the people friendly aspects.
2015	Please don't allow a mix of the lovely older and often stately homes with some very futuristic and totally without soul new homes. New houses should at least try to blend in with existing houses. Some of the new houses in Old Strathcona scream to the heavens, they are THAT ugly.
2016	I would like them to stay the same.  These regulations were the result of a lot of thought and negotiation and are a good compromise.  Perhaps some additional density would be okay, but skinny houses on 50 foot lots are ridiculous.  Duplexes maybe; skinny houses on 60 foot lots okay.
3001	When people are helpless I get angry -- at them, at systemic injustice, at the scale of challenges facing society. Seeing people at their worst is bewildering and frightening.
3002	Education should be at the heart of culture, aligned with the natural principles of the cosmos allowing for the flowering of self, community, and world systems.
3003	I feel sorry for people who are failed by our society and suffer as a result of circumstances, lack of opportunity and inability to take advantage of opportunities.
3004	My co-workers and I arise together, knowing and not knowing, roles arising and falling away, out of which life itself creates anew, carrying our dance like seeds in a breeze.
3005	What gets me into trouble is Falling from eternity and spaciousness into contraction and then forgetting that this is also eternity manifesting sometimes as frustration and judgement.
3006	When I get mad i share my view so that I can begin to see what I am missing and not seeing. I enter into dialogue to move into reflection, possibilities, and action if needed.
3007	Rules constantly evolve to give integrity to each developmental stage, and yet are relative within the larger chaotic openness that is the unfolding nature of reality.
3008	We could make the world a better place if could spread love and compassion to all, bring happiness to everyone we meet. If everyone would show a little respect and take responsibilities for there actions.
3009	If my mother Hadnt died through a drawn out heart breaking decline into dementia, I would never have loved her as fully as I do now. Why are we always looking up to the light?
3010	I just can\'t stand people who who get their racism incorrect. Then I don\'t know if I should get mad for getting misidentified as a Pilipino or feel sorry for the guy who does not know world map.
3011	If I were in charge Of anything right now it would be a challenge because I do not seem to be able to believe in the idea that anyone can actually BE in charge. It is a mutual illusion.
3012	These days progress means letting go while driving forward; finding control in surrender; enjoying connection in autonomy; and playing with too many other polarities and paradoxes to count...
3013	Sometimes I wish that the story would be just a little looser, that I could slide story on when I needed it and let it slip off when I didn't, walk around as the universe and as Levi freely.
3014	I am a mother, a wife, a daughter, a sister,a friend,a boss , a coworker, a teacher, a lobbyist, a humanitarian person, helping the most people i can, a citizen of this world
3015	My main problem is at the moment, that Im hesitant, almost procrastinating, with regards to a new project dear to me in which I would have to step out and become more visible to the public.
3016	I just can\'t stand people who I just can't stand people who stand people on just I and then I can't stand the people I am sometimes and when I stand as I and can't people myself out of the person I am.
3017	My co-workers and I Work very well together as we have a long history of successes and failures and thus a solid foundation of trust and a shared vision of what doing a great job really means.
3018	If my mother was alive, I would love to talk with her, share more with her, not be scared of her criticism and rejection. I would love to get to a warm loving respectful space with her.
3019	When people are helpless they either do not have, or have temporarily lost the truth of the vastness of wisdom and agency available to them, even if this means that there is much sacrifice involved.
3020	Being with other people can be joyful, stimulating, healing, or it can be complex, thorny and challenging. My experience around people is very dependent on the quality of my relationships with them.
3021	When people are helpless let it be a moment in time a need defined a still point for some new turning a siren heard and felt in the fabric of our interbeing a call and response that belongs to us all
3022	My main problem is I don't have the discipline that I need to succeed until I become fixated on a goal and even then it isn't the end that I am working for its the failure I am working to avoid.
3023	What I like to do best is being in flow with whatever I do, a state of blissful purposeful presence to the task at hand, being it walking on the beach, riding my bike or doing clinical work or teaching.
3024	Privacy is something which is required at times and sometimes its a form of withdrawal and its important to know whether integrity is bringing this about or an ego contraction or fear.
3025	These days, teaching is a very demanding profession, and in my experience teachers are generally overextended unless they are well organized, and they need better training. preparation, and support.
3026	The past seems like remembering a movie I once watched, one that I can have feelings about when remembered...and more feelings when there's something i haven't yet digested from the past
3027	The thing I like about myself is how radiant and resplendent the blessing of creation's unfurling is, that I have for these breaths this incarnate, particular, sensory dip in the ocean of all-beingall-becoming.
3028	These days, work is a word I relate to less and less. Every moment there is what the body is doing and there is awareness. In awareness the distinctions of work and play and rest seem insignificant.
3029	If I had more money I'd live with different opportunities that would, or may not, open me more, or less..... for now, what is is, and I'm happy with the providence that bathes me in joyous interactions.
3030	When I am criticized a process of defense, consideration, values, perspective, and response occurs - sometimes it takes 500 milliseconds and sometimes it takes a little longer. The process can be cyclical.
3031	When people are helpless they need support to move forward. "Helpless" does not need to be a permanent descriptor, most people are able to be supported to re-direct into a position which plays to their strengths.
3032	What I like to do best is Meditate and rest in bliss. In that place and space there is nothing ever missing and the mind can finally stop seeking and just shut up and be. I also enjoy snuggling my children and cat.
3033	When they avoided me I wondered is this mine or theirs? and feeling the contraction, sometimes I doubted myself as \"good\" and tried to muster the courage to ask them about it while scared to hear their reply.
3034	Privacy is about avoiding the fear of exposing ourselves &#x28;or others&#x29; to the judgment of others - it's probably not a 'need', but a 'want' that has become an expectation in western society.
3035	What I like to do best is be here in this moment with all parts of me, here, in the unknowable known moment and breathe, be being by being, feel the inseparability of the me and the world, the environment and my mind.
3036	We could make the world a better place if Oh, God better not to ask the question, and rather turn our attention in a way that allows Compassion to overtake us so fully that the world and self dissolve in light, and we can laugh again.
3037	What gets me into trouble is Fear. That's the first thing to come to mind. When faced with a potential conflict, I will either fight or freeze, and each of these get me into trouble. I also think I lack a certain emotional intelligence in real time, so I don't realize that I should employ skillful means to communicate my vision or position. Instead I just foolishly move forward in a politically unwise fashion.
3038	My father is caring, sometimes nice, I'm lucky to have a dad, a dad is like amor he cares for you, sometimes like a grandpa and grandma, but even more important, he is part of you he helped make you so a dad is something lucky to have. You couldn't even be alive if you didn't have a dad, well you could if he died but then you don't have a dad, but you did have a dad which means you do have a dad.
3039	Change is happening all the time, continues and is inevitable. Change for each of us and as a species is product our individual and collective memory of our knowledge and experiences through time. Sometimes the change is thought to be external to ourselves, sometimes it is viewed to be caused by us individually. In some respects change is an illusion of our understanding and experiencing of time.
3040	What gets me into trouble is My intellect. My mind is ultra fast and If im not fully in my center, I sometimes jap out something stupid. Because I easily see peoples struggle I can also easily hurt them.. Its a defence programme. It hasn't come up for a while now and I assume I got the better of it. There are many other things that will get me in trouble if I dont attend to my inner self in the for of my spiritual practice
3041	These days, work from the perspective of the individual, is to reach beyond the confines of the mind, heart and body and use them to sense oneness with the gross, subtle and causal. From the perspective of the one, work is to sense through the specific circumstances of the individual to see where healing is required in the ever-present now and as mentioned above to apply to that location the energy of love and life.
3042	When I am criticized I find myself going inward to first examine my response to this criticism, and also the state of my heart. I live from a commitment to have an open mind to the feedback which I receive from the world , with which I am engaged and to be teachable and willing to listen, so that I might learn from others. I acknowledge as well the pain which such criticism creates within the, and allow this pain to teach me as well.
3043	These days, work is a giving back to the seeds planted by the Kosmic ocean. Through the exchange of the infinite Delight it moves through the \"me\" and asks for an unfolding of what it give to offer back to the whole. Work is also the causal ocean \"out there\" moving and calling forth itself from within this swirl, this formMy \"i\" and work is constituted by the Whole through all layers and levels of concrete, subtle and causal.
3044	Raising a family I salute all those who are raising families and I know that this goes beyond the parents and it takes a village to raise a family - this feels more true these days and I am grateful for lifetimes of families and generations of families and ancestors who have played a part in raising me. While I will not be raising my own family,I do feel that I contribute to raising individuals by my presence and interaction with them.
3045	When I am criticized it is one face of divinity noticing separation and responding. Every movement is a movement of an Ocean reaching into concrete, sublet and causal forms. it swirls at different rates which hold different noticingwe see what we have not loved and embraced as the ocean of delight. When someone see my form and shape and criticizes it, it is the divine looking at me and emendating its beauty towards itself in the collective.
3046	Love changes when First, when there is a connectiononeness that unites consciousness, heart and body so that the localized and the infinite are experienced simultaneously. And second, when one feels the ability to shift the cosmic flow of life and love and direct some of its healing power where it is needed, either within the deepest essence of a loved one, a person in need, toward a global tragedy, or a world view or meme that is doing harm.
3047	I am the voice at the edge of hearing, the movement in the corner of the eye, the novelty that makes you smile, the light in the darkness, and the nothingness at the heart of it all. I am the string, the wave, the field, the packet of energy. I am the watcher, the smiler, the laughter and the lover. I am the non-doer, the finger flowing through the air. The symbol and the sign. I am the deep belly laugh that brings forth existence.
3048	When I am criticized I can have a range of responses depending upon how tired or resourceful I am or how closely what amnd how they say it matches a previous wound trigger. I can react defensively or just ask why they say that or as I often do ask them to "say more" so that I can more completely understand what they are saying so that I can fully understand their point of view, the context from which they formulated their thought and learn from it.
3049	Business and society need a new social contract. They coexist with a great deal of tension in a capitalist structure and shareholders win out at the expense of the workers, the environment, and the community at large. Business needs to be in the \"business\" of generating value not wealth and operating within the constraints of our natural environment or actually contributing to better stewardship of the resources we have so we dont destroy ourselves.
3050	When I am criticized I know there is something I must have missed, something I haven\'t noticed or a dynamic at work that is causing the person criticizing me to see in me a gap of some sort. And while that gap may or may not be there in reality, it is in that person\'s mind. This provides me the opportunity to explore that person\'s mind or world view. I would like that exploration to result in growth for us both or at least to enhance our relationship.
3051	Crime and delinquency could be halted if we could see that the essence of who we are in unity and in variation are two side of one coin of WholnessIF we could see all \"other\" as our very own self, we could become privy to a Love that heals, that gently embraces the ways an Eternity postures and veils itself as a separation. in this embrace, the separation melts, because God is embraced at any level it shows its beautiful facein that embrace we feel heard and we feel whole...
3052	If my mother I feel a pull to look back at my own upbringing and my mother's inevitable influence on my development. I am also aware of having moved beyond dwelling on unfinished business and see how much a trap that is for the ego and my identifying with the unmet need. I suppose if my mother had been different I might have had different hang ups, but as I believe now I am the only one who can do anything about them, it feels a bit immaterial right now.
3053	These days, work Is something I continue to enjoy and take great pleasure in using it as a framework within which to continue my journey of discovery and fulfilment. Work is not separate to life. Work is part of living, of serving, of being useful. Indeed, one could call work - play. There is no separation. Whether I receive the appropriate value recognition or remuneration for my "job" is my judgement. Voluntary work, while unpaid, is the most rewarding, I find.
3054	Change is the shittiest fucking thing ever; I so wish reality was completely static sometimes... at least, so says some part of me, at least right now - the thirst for permanence comes in waves, and it's pretty strong in me today... did I mention I'm wrestling with a couple of big and difficult life changes at the moment? :-) This is not representative of how I usually respond to this stem or talk about change, it's just what I'm present to at the moment...
3055	Being with other people Is being nested within the eternal field that moves through and within everything. Charged as though all animate and inanimate forms are breathing, being breathed by the causal whole, so too, being with other people is being with all forms and the divine currents that extend Everywhere. It is like feeling and seeing the Divine Eternity at eternal play within itself; the One through the many as we all dance as fullness, nested within a eternal expanse...
3056	My main problem is being in my head rather than fully present and instead running trauma patterns habitsbeliefs of being a victim, feeling I need to please others and abandon myself and then be angry when not cared for by others. The other main problem is not fitting in to the "popular Culture" world as I am often "too deep" seeing metaphorical symbolism and duplicityinauthenticity all around me and wanting to serve and be one with spirit rather then pursuing material things.
3057	Privacy What can universal consciousness possibly conceive as 'private'...big laugh :) Still laughing :) :) This is hilarious. Everything is seen and is 'it' all of consciousness. In the localised expression of consciousness (as humans) at some levels and stages of development this is a great concern as it shows the humanity of fear, power, manipulation etc, all of which our evolution conspires to create (and also withhold its potential for unfolding). And this is so.
3058	Change is ...happening each day on the one side: kids grow, you realise you get older and remember less, circumstances of living are different today, then yesterday. So you need to \"change\" your plan of today, you need to be open and flexibel and adopt to it, for example. Changes of behaviour or values demand go much depper and need to be reflected first. It can enter in new developments, but it is difficult to achieve this - personal as well as from the perspective of a society.
3059	My main problem is trying to \'wrestle with the angel\'. The Vision is so Vast, there is no way that my human mind can \'wrap itself around it\'. It doesn\'t know how. Interestingly, my virtual assistant, Melanie Davis, whom I\'ve never met \'face to face\', brought to my awareness the Mission of Called By Love Institute when she said that the two people who are part of our core team working on building the new website seem to be doing it! The Mission calls it \'Harnessing the Power of Love\'
3060	My co-workers and I My co-workers in the evolution of consciousness are members of 3 weekly weekly generative dialog groups. Together we enter shared containers of collective consciousness and collective heart. We explore who we are in this space and what is possible. We let go of our individual (subtle) egos and we practice trying on and owning identities grounded in more expansive perspectives. The ultimate goal is to share a planetary or Kosmic identity and once aligned, see what we can accomplish together as one.""")

df = pd.read_csv(tsv, delimiter = "\t")
print(df.columns)
df.sample(frac=1).head()

Index(['tempID', 'sentence'], dtype='object')


,tempID,sentence
58,59,I am often confused about parts of speech.
125,1038,When I am criticized I can embody my personali...
124,1037,My conscience bothers me if I'm not placing my...
122,1035,We could make the world a better place if the ...
82,83,How do you do?


In [5]:
import re, html
PATTERN = "[^a-zA-Z0-9\s]+"
rgx = re.compile(PATTERN, re.IGNORECASE)

df['preprocessed_sentence'] = df['sentence'].apply(lambda ip : re.sub('\s+', ' ', rgx.sub(' ', html.unescape(ip))))
print(df.columns)
df.sample(frac=1).head()

Index(['tempID', 'sentence', 'preprocessed_sentence'], dtype='object')


,tempID,sentence,preprocessed_sentence
160,3008,We could make the world a better place if coul...,We could make the world a better place if coul...
184,3032,What I like to do best is Meditate and rest in...,What I like to do best is Meditate and rest in...
183,3031,When people are helpless they need support to ...,When people are helpless they need support to ...
28,29,That building is a mine.,That building is a mine
17,18,I have a vacuum cleaner.,I have a vacuum cleaner


In [6]:
df['nlp_doc'] = df['preprocessed_sentence'].apply(lambda ip : nlp(ip))
print(df.columns)
df.sample(frac=1).head()

Index(['tempID', 'sentence', 'preprocessed_sentence', 'nlp_doc'], dtype='object')


,tempID,sentence,preprocessed_sentence,nlp_doc
154,3002,"Education should be at the heart of culture, a...",Education should be at the heart of culture al...,"(Education, should, be, at, the, heart, of, cu..."
171,3019,When people are helpless they either do not ha...,When people are helpless they either do not ha...,"(When, people, are, helpless, they, either, do..."
51,52,I will answer for myself.,I will answer for myself,"(I, will, answer, for, myself)"
55,56,We ought to let him know he needs to eat prope...,We ought to let him know he needs to eat prope...,"(We, ought, to, let, him, know, he, needs, to,..."
163,3011,If I were in charge Of anything right now it w...,If I were in charge Of anything right now it w...,"(If, I, were, in, charge, Of, anything, right,..."


Total % of sentences with correct reconstructions =  0.81 . It's actually greater than 81% since complex first clauses followed by conjunctions put the conjuction with the parent clause in the first.

In [21]:
def flatten_list(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list

def get_children(doc):
    if len([x for x in doc.children]) == 0:
        return [doc]
    if doc.pos_ == "VERB":
        return []

    op = flatten_list([get_children(l) for l in doc.lefts]) + [doc] + flatten_list([get_children(r) for r in doc.rights])
    return op

def postprocess(tokens_arr):
    if len(tokens_arr) == 1 and ( tokens_arr[0].dep_ in ["aux", "auxpass"] or tokens_arr[0].tag_ in ["VBG"]): 
        return []
    return tokens_arr

def get_text_from_tokens(tokens_arr):
    op = ' '.join([x.text for x in tokens_arr])
    op = op.replace(" nt", "nt").replace(" '", "'")
    return op

def clause_split_by_verbs(doc):
    op = []
    #print("#####################################################################")
    for token in doc:
        #print(token.text, token.tag_, token.dep_, token.pos_)
        if token.pos_ == "VERB":
            arr = flatten_list([get_children(l) for l in token.lefts]) + [token] + flatten_list([get_children(r) for r in token.rights])
            #print(arr)
            arr = postprocess(arr)
            txt = get_text_from_tokens(arr)
            if txt.strip() != '':
                op.append(txt)
    # No verb in the text provided like "Oh wow!"
    if len(op)==0:
        op.append(doc)
    return op#' '.join(["|{}| {}".format(i+1, op[i]) for i in range(len(op))])

df['split_by_verbs_arr'] = df['nlp_doc'].apply(clause_split_by_verbs)
df['split_by_verbs'] = df['split_by_verbs_arr'].apply(lambda op : ' '.join(["|{}| {}".format(i+1, op[i]) for i in range(len(op))]))
print(df.columns)
df[['tempID', 'sentence', 'split_by_verbs']].to_csv("./output.csv", index = False)

df['reconstructed'] = df['split_by_verbs'].apply(lambda x : re.sub("\s+", " ", re.sub("\|\d+\|", "", x)))
df['is_correct'] = df.apply(lambda x : re.sub("[a-z0-9]+", "", x['preprocessed_sentence'].strip().lower()) == re.sub("[a-zA-Z0-9]+", "", x['reconstructed'].strip().lower()), axis = 1)
df[['preprocessed_sentence', 'reconstructed', 'is_correct']].sample(frac=1).head()


Index(['tempID', 'sentence', 'preprocessed_sentence', 'nlp_doc',
       'split_by_verbs_arr', 'split_by_verbs', 'reconstructed', 'is_correct'],
      dtype='object')


,preprocessed_sentence,reconstructed,is_correct
181,If I had more money I d live with different op...,If I had more money I d live with different o...,True
160,We could make the world a better place if coul...,We could make the world a better place if cou...,True
54,What shall we say to him about his behavior at...,What shall we say to him about his behavior a...,True
109,Technology is a great tool nice to have and ca...,Technology is a great tool nice to have and c...,False
140,Totally against densification because it keeps...,because it keeps the fire department too busy,False


In [22]:
df[['sentence', 'split_by_verbs']].sample(frac = 10/df.shape[0])

,sentence,split_by_verbs
40,That car is mine.,|1| That car is mine
85,That's part of what it takes to succeed.,|1| That s part of |2| what it takes |3| to su...
53,You must eat your vegetables.,|1| You must eat your vegetables
113,If I were in charge i would change that by fac...,|1| If I were in charge |2| i would change tha...
100,Being with other people is both a learning exp...,|1| Being with other people |2| is both a lear...
148,"Once regulations are in place, they should be ...",|1| Once regulations are in place |2| they sho...
67,I felt bad for her.,|1| I felt bad for her
190,"My father is caring, sometimes nice, I'm lucky...",|1| My father is caring sometimes nice |2| I m...
108,Rules are guard rails. It is good and helpful ...,|1| Rules are guard rails |2| It is good and h...
133,When I get mad i feel adrenaline through my hu...,|1| When I get mad |2| i feel adrenaline throu...


In [20]:
df[df.is_correct].shape[0]/df.shape[0]

0.812206572769953